In [73]:
import ecole
import shutil
import os
import sys
import time
import numpy as np
from tqdm import trange
import gurobipy as gb
import pyscipopt as pyscip
from gurobipy import GRB
import pickle as pkl

from dump_data import *

scip_parameters = {"branching/scorefunc": "s",
                   "separating/maxrounds": 0,
                   "limits/time": 360,
                   "presolving/maxrounds": 0,
#                    "constraints/linear/singletonstuffing": False,
#                    "constraints/linear/rangedrowartcons": False,
#                    "constraints/and/maxprerounds" :0,
#                    "constraints/and/presolusehashing": False,
#                    "constraints/benders/maxprerounds": 0,
#                    "constraints/benderslp/iterlimit": 0,
#                    "reading/dynamicrows": False,
#                    "separator/oddcycle/cutthreshold":0,
                  }

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout


# Generate Problem and collect features

In [71]:
if os.path.exists("DataSet/"):
    shutil.rmtree("DataSet/")
os.mkdir("DataSet/")

env = ecole.environment.Branching(\
            observation_function = ecole.observation.NodeBipartite(),scip_params = scip_parameters)

for row in [100,200,300,400,500]:
    for coef_col in [1,1.5,2]: 
        for d in [0.1,0.15,0.2]:
            col = int(coef_col * row)
            setCover = ecole.instance.SetCoverGenerator(n_rows = row, n_cols = col,density = d) 
            print("Generate with Row:%d,Col:%d,Density:%f" % (row,col,d))
            for n in trange(1,101):
                done = False
                while(not done):
                    try:
                        problme_name = \
                            "set_cover_{"+row.__str__()+"*"+col.__str__()+"_"+d.__str__()+"_"+n.__str__()+"}"
                        instance = next(setCover)
                        #save problm lp
                        problme_name = \
                            "set_cover_{"+row.__str__()+"*"+col.__str__()+"_"+d.__str__()+"_"+n.__str__()+"}"
                        os.mkdir("DataSet/"+problme_name)
                        instance.write_problem("DataSet/"+problme_name+"/problem.lp")
                        #save features
                        obs, _, _, _, _ = env.reset(instance)
                        dumpRowFeatures("DataSet/"+problme_name+"/constraints_features.json",obs.row_features)
                        dumpVariableFeatures("DataSet/"+problme_name+"/variables_features.json",obs.variable_features)
                        dumpEdgeFeatures("DataSet/"+problme_name+"/edges_features.pkl",obs.edge_features,row,col)
                        done = True             
                    except Exception as ex:
#                         print("Erreur:%s"%ex)
                        done = False
                        shutil.rmtree("DataSet/"+problme_name)
            

Generate with Row:100,Col:100,Density:0.100000


100%|█████████████████████████████████████████| 100/100 [00:04<00:00, 24.13it/s]


# Collect labels

In [72]:
if os.path.exists("DataSet/label/"):
    shutil.rmtree("DataSet/label/")
os.mkdir("DataSet/label/")
gapList = []
for row in [100]:
    for coef_col in [1]: 
        for d in [0.1]:
            col = int(coef_col * row)
            print("Solve with Row:%d,Col:%d,Density:%f" % (row,col,d))
            for n in trange(1,101):
#                 #save label
                with HiddenPrints():
                    problme_name = \
                            "set_cover_{"+row.__str__()+"*"+col.__str__()+"_"+d.__str__()+"_"+n.__str__()+"}"
#                     solver = gb.read("problem/"+problme_name+".lp")
#                     solver.optimize()
#                     dumpSolution_Gurobi("DataSet/label/"+problme_name+"_label.json",solver)
                    solver = ecole.scip.Model.from_file("DataSet/"+problme_name+"/problem.lp")
                    aspyscip = solver.as_pyscipopt()
                    aspyscip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
                    aspyscip.optimize()
                    gapList.append(aspyscip.getGap())
                    dumpSolution_Ecole("DataSet/"+problme_name+"/label.json",aspyscip)
                    
gap = np.array(gapList)
print(np.mean(gap))
print(np.where(gap != 0))

Solve with Row:100,Col:100,Density:0.100000


  0%|                                                   | 0/100 [00:00<?, ?it/s]


NameError: name 'HiddenPrints' is not defined

In [ ]:
#scip
with HiddenPrints():
    model_scip = pyscip.scip.Model()
    model_scip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
    model_scip.readProblem(filename="debugFile/set_cover_test.lp",extension = "lp")
    # model_scip.readProblem(filename="debugFile/debug_diffc.lp",extension = "lp")

model_scip.optimize()
# time.sleep(1)
print("\n",model_scip.getBestSol())

# print("Ecole*********************")
# # with HiddenPrints():
# solver = ecole.scip.Model.from_file("debugFile/debug.lp")
# aspyscip = solver.as_pyscipopt()
# aspyscip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
# aspyscip.optimize()
# gap = aspyscip.getGap()
# sol = aspyscip.getBestSol()
# print(sol,gap)

# #gurobi
# print("Gurobi*********************")
# with HiddenPrints():
#     model = gb.read("debugFile/debug.lp")
#     model.optimize()
# for var in model.getVars():
#     print(f"{var.varName}: {round(var.X, 3)}")



In [7]:
import ecole
import shutil
import os
import sys
import time
import numpy as np
from tqdm import trange
import gurobipy as gb
import pyscipopt as pyscip
from gurobipy import GRB
import pickle as pkl

from dump_data import *

scip_parameters = {"branching/scorefunc": "s",
                   "separating/maxrounds": 0,
                   "limits/time": 360,
                   "presolving/maxrounds": 0,
                   "separating/maxroundsroot" : 0,
                   "separating/maxcutsroot": 0,
                   "separating/maxcuts": 0,
                   "propagating/maxroundsroot" : 0,
                   "lp/presolving" : False,
                   "constraints/linear/sortvars": False,
                   "heuristics/shiftandpropagate/sortvars": False,
                   "propagating/genvbounds/sort": False,
                   "randomization/permutevars": False,
                   "randomization/permuteconss": False,
                   "presolving/gateextraction/sorting":0,
                   "separating/oddcycle/sortswitch":0,
#                    "constraints/linear/maxrounds" :0,
#                    "constraints/linear/maxroundsroot" : 0,
#                    "constraints/knapsack/maxroundsroot" : 0,
#                    "separating/maxroundsrootsubrun" : 0,
#                    "separating/zerohalf/maxroundsroot" : 0,
#                    "separating/rlt/maxroundsroot" : 0,
#                    "separating/oddcycle/maxroundsroot" : 0,
#                    "separating/mixing/maxroundsroot" : 0,
#                    "separating/minor/maxroundsroot" : 0,
#                    "separating/interminor/maxroundsroot" : 0,
#                    "separating/gomory/maxroundsroot" : 0,
#                    "separating/eccuts/maxroundsroot" : 0,
#                    "separating/disjunctive/maxroundsroot" : 0,
#                    "separating/aggregation/maxroundsroot" : 0,
                   
#                    "constraints/linear/singletonstuffing": False,
#                    "constraints/linear/rangedrowartcons": False,
#                    "constraints/and/maxprerounds" :0,
#                    "constraints/and/presolusehashing": False,
#                    "constraints/benders/maxprerounds": 0,
#                    "constraints/benderslp/iterlimit": 0,
#                    "reading/dynamicrows": False,
#                    "separator/oddcycle/cutthreshold":0,
                  }
done = 0
while not done:
    try:
        setCover = ecole.instance.SetCoverGenerator\
                    (n_rows=10, n_cols=10,density=0.3)
        env = ecole.environment.Branching(observation_function = ecole.observation.NodeBipartite(),\
                                        scip_params = scip_parameters)
        instance = next(setCover)
#         print(instance)
        problme_name = "set_cover_test"
        instance.write_problem("debugFile/"+problme_name+".lp")

        obs, action_set, _, _, _ = env.reset(instance)
        print(obs.row_features.shape)
        print(obs.edge_features.shape)
        print(obs.variable_features.shape)
        done = 1
    except:
        pass
    

# setCover = ecole.instance.SetCoverGenerator\
#             (n_rows=300, n_cols=300,density=0.1)
# env = ecole.environment.Branching(observation_function = ecole.observation.NodeBipartite(),\
#                                 scip_params = scip_parameters)
# instance = next(setCover)
# print(instance)
# problme_name = "set_cover_test"
# instance.write_problem("debugFile/"+problme_name+".lp")

(10, 5)
[10, 10]
(10, 19)


In [4]:
dumpEdgeFeatures("debugFile/test_FE.json",obs.edge_features,150,150)


In [41]:
#A
for i in range(len(obs.edge_features.values)):
    edges[obs.edge_features.indices[0][i]][obs.edge_features.indices[1][i]] = obs.edge_features.values[i]
print(edges)

[[ 0.         -0.57735027  0.         -0.57735027 -0.57735027  0.
   0.          0.          0.          0.        ]
 [-0.5         0.          0.          0.          0.         -0.5
  -0.5         0.          0.         -0.5       ]
 [-0.70710678  0.          0.         -0.70710678  0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.         -0.70710678  0.          0.
   0.         -0.70710678  0.          0.        ]
 [ 0.         -0.5        -0.5         0.          0.         -0.5
  -0.5         0.          0.          0.        ]
 [ 0.          0.         -0.5        -0.5         0.          0.
   0.         -0.5         0.         -0.5       ]
 [ 0.          0.         -0.57735027  0.          0.          0.
  -0.57735027  0.         -0.57735027  0.        ]
 [ 0.         -0.70710678  0.          0.         -0.70710678  0.
   0.          0.          0.          0.        ]
 [ 0.         -0.57735027  0.          0.         -0.57735027 

In [43]:
#bias
print(obs.row_features[:,0])

[-0.57735027 -0.5        -0.70710678 -0.70710678 -0.5        -0.5
 -0.57735027 -0.70710678 -0.57735027 -0.57735027]


In [79]:
edges_dict = {}
for i in range(len(obs.edge_features.indices[0])):
    if obs.edge_features.indices[0][i] not in edges_dict:
        edges_dict[obs.edge_features.indices[0][i]] = []
    edges_dict[obs.edge_features.indices[0][i]].append(obs.edge_features.indices[1][i])
np.set_printoptions(threshold=np.inf)
print(edges_dict)

{0: [3, 4, 6], 1: [2, 5, 7, 9], 2: [0, 1, 2], 3: [0, 3, 6], 4: [0, 5, 7], 5: [2, 8], 6: [0, 5, 9], 7: [0, 1, 4, 7], 8: [3, 5, 8], 9: [4, 5]}


In [70]:
scip_parameters = {"branching/scorefunc": "s",
                   "separating/maxrounds": 0,
                   "limits/time": 360,
                   "presolving/maxrounds": 0,
                   "separating/maxroundsroot" : 0,
                   "separating/maxcutsroot": 0,
                   "separating/maxcuts": 0,
                   "propagating/maxroundsroot" : 0,
                   "lp/presolving" : False,
                   "constraints/linear/sortvars": False,
                   "heuristics/shiftandpropagate/sortvars": False,
                   
                   "propagating/genvbounds/sort": False,
                   "presolving/sparsify/rowsort": False,
                   
                   "randomization/permutevars": False,
                   "randomization/permuteconss": False,
                   
                   "presolving/gateextraction/sorting":0,
                   "separating/oddcycle/sortswitch":0,
                   "separating/oddcycle/sortrootneighbors":False,
                   
#                    "constraints/linear/maxrounds" :0,
#                    "constraints/linear/maxroundsroot" : 0,
#                    "constraints/knapsack/maxroundsroot" : 0,
#                    "separating/maxroundsrootsubrun" : 0,
#                    "separating/zerohalf/maxroundsroot" : 0,
#                    "separating/rlt/maxroundsroot" : 0,
#                    "separating/oddcycle/maxroundsroot" : 0,
#                    "separating/mixing/maxroundsroot" : 0,
#                    "separating/minor/maxroundsroot" : 0,
#                    "separating/interminor/maxroundsroot" : 0,
#                    "separating/gomory/maxroundsroot" : 0,
#                    "separating/eccuts/maxroundsroot" : 0,
#                    "separating/disjunctive/maxroundsroot" : 0,
#                    "separating/aggregation/maxroundsroot" : 0,
                   
#                    "constraints/linear/singletonstuffing": False,
#                    "constraints/linear/rangedrowartcons": False,
#                    "constraints/and/maxprerounds" :0,
#                    "constraints/and/presolusehashing": False,
#                    "constraints/benders/maxprerounds": 0,
#                    "constraints/benderslp/iterlimit": 0,
#                    "reading/dynamicrows": False,
#                    "separator/oddcycle/cutthreshold":0,
                  }
env = ecole.environment.Branching(observation_function = ecole.observation.NodeBipartite(),\
                                scip_params = scip_parameters)
instance = ecole.scip.Model.from_file("debugFile/set_cover_test.lp")
print(instance)

obs, action_set, _, _, _ = env.reset(instance)
print(obs.row_features.shape)
print(obs.edge_features.shape)
print(obs.variable_features[:,0])

edges_dict = {}
for i in range(len(obs.edge_features.indices[0])):
    if obs.edge_features.indices[0][i] not in edges_dict:
        edges_dict[obs.edge_features.indices[0][i]] = []
    edges_dict[obs.edge_features.indices[0][i]].append(obs.edge_features.indices[1][i])
np.set_printoptions(threshold=np.inf)
print(edges_dict)
edges = np.zeros((10,10))

(10, 5)
[10, 10]
[0.55795062 0.04601655 0.21282652 0.27609928 0.1265455  0.14955377
 0.49467787 0.4889258  0.00575207 0.20707446]
{0: [5, 9], 1: [0, 3, 7], 2: [0, 3, 6, 9], 3: [1, 2, 4], 4: [1, 6], 5: [0, 5], 6: [0, 2, 4, 8], 7: [0, 3, 5, 8], 8: [1, 4, 8], 9: [0, 7, 9]}


In [ ]:
for row in [100,150,200]:
    for coef_col in [1,1.5,2]: 
        for d in [0.1,0.15,0.2]:
            col = int(coef_col * row)
            print("Solve with Row:%d,Col:%d,Density:%f" % (row,col,d))
            for n in range(1,2):
#                 #save label
                problme_name = \
                        "set_cover_{"+row.__str__()+"*"+col.__str__()+"_"+d.__str__()+"_"+n.__str__()+"}"
#                     solver = gb.read("problem/"+problme_name+".lp")
#                     solver.optimize()
#                     dumpSolution_Gurobi("DataSet/label/"+problme_name+"_label.json",solver)
#                 solver = ecole.scip.Model.from_file("DataSet/problem/"+problme_name+".lp")
#                 aspyscip = solver.as_pyscipopt()
#                 aspyscip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
#                 aspyscip.optimize()
                model_scip = pyscip.scip.Model()
                model_scip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
                model_scip.readProblem(filename="DataSet/problem/"+problme_name+".lp")
                model_scip.optimize()